# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

- **Consider a significance level of 5% for all tests.**

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as st

# Challenge 1 - Independent Sample T-tests

In this challenge, we will be using the Pokemon dataset. Before applying statistical methods to this data, let's first examine the data.

To load the data, run the code below.

In [3]:
# Run this code:

pokemon = pd.read_csv('../pokemon.csv')

Let's start off by looking at the `head` function in the cell below.

In [4]:
pokemon.head()

,#,Name,Type 1,Type 2,Total,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Generation,Legendary
0,1,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,3,VenusaurMega Venusaur,Grass,Poison,625,80,100,123,122,120,80,1,False
4,4,Charmander,Fire,NaN,309,39,52,43,60,50,65,1,False


The first thing we would like to do is compare the legendary Pokemon to the regular Pokemon. To do this, we should examine the data further. What is the count of legendary vs. non legendary Pokemons?

In [5]:
pokemon.groupby('Legendary').agg({'Legendary':'count'})

,Legendary
Legendary,
False,735
True,65


Compute the mean and standard deviation of the total points for both legendary and non-legendary Pokemon.

In [6]:
mean_leg = pokemon[pokemon['Legendary']==True]['Total'].mean()
std_leg = pokemon[pokemon['Legendary']==True]['Total'].std(ddof=1)

mean_non_leg = pokemon[pokemon['Legendary']==False]['Total'].mean()
std_non_leg = pokemon[pokemon['Legendary']==False]['Total'].std(ddof=1)

print('Legendaries mean and st: ', mean_leg,',', std_leg)
print('Non legendaries mean and st: ', mean_non_leg,',', std_non_leg)

Legendaries mean and st:  637.3846153846154 , 60.93738905315344
Non legendaries mean and st:  417.21360544217686 , 106.76041745713005


The computation of the mean might give us a clue regarding how the statistical test may turn out; However, it certainly does not prove whether there is a significant difference between the two groups.

In the cell below, use the `ttest_ind` function in `scipy.stats` to compare the the total points for legendary and non-legendary Pokemon. Since we do not have any information about the population, assume the variances are not equal.

In [7]:
# 1) 

# H0: mu total stats of Legendary pokemon = mu total stats of non-Legendary pokemon
# H1: mu total stats of Legendary pokemon != mu total stats of non-Legendary pokemon

# Double tailed

# 2)

alpha = 0.05

# 3) Sample

legendaries = pokemon[pokemon['Legendary']==True]['Total']
non_legendaries = pokemon[pokemon['Legendary']==False]['Total']

# 4 and 5

stat, p_value = st.ttest_ind(legendaries, non_legendaries, equal_var=False)

print(stat)
print(p_value) 

25.8335743895517
9.357954335957446e-47


What do you conclude from this test? Write your conclusions below.

In [8]:
# The p_value is smaller than alpha, we can reject H0 and state that 
# the mu total stats of legendary pokemon will be more likely different from 
# the non-legendary ones.


How about we try to compare the different types of pokemon? In the cell below, list the types of Pokemon from column `Type 1` and the count of each type.

In [10]:
pokemon.groupby('Type 1').agg({'Type 1':'count'})



,Type 1
Type 1,
Bug,69
Dark,31
Dragon,32
Electric,44
Fairy,17
Fighting,27
Fire,52
Flying,4
Ghost,32


Since water is the largest group of Pokemon, compare the mean and standard deviation of water Pokemon to all other Pokemon.

In [12]:
mean_water = pokemon[pokemon['Type 1'] == 'Water'].mean()
std_water = pokemon[pokemon['Type 1'] == 'Water'].std(ddof=1)
mean_rest = pokemon[pokemon['Type 1'] != 'Water'].mean()
std_rest = pokemon[pokemon['Type 1'] != 'Water'].std(ddof=1)

print('Mean of water pokemon: ', mean_water, 'mean of the rest of the pokemon: ', mean_rest)
print('Std of water pokemon: ', std_water, 'std of the rest of the pokemon: ', std_rest)


Mean of water pokemon:  #             303.089286
Total         430.455357
HP             72.062500
Attack         74.151786
Defense        72.946429
Sp. Atk        74.812500
Sp. Def        70.517857
Speed          65.964286
Generation      2.857143
Legendary       0.035714
dtype: float64 mean of the rest of the pokemon:  #             372.536337
Total         435.859012
HP             68.802326
Attack         79.790698
Defense        73.988372
Sp. Atk        72.495640
Sp. Def        72.127907
Speed          68.654070
Generation      3.399709
Legendary       0.088663
dtype: float64
Std of water pokemon:  #             188.440807
Total         113.188266
HP             27.487026
Attack         28.377192
Defense        27.773809
Sp. Atk        29.030128
Sp. Def        28.460493
Speed          23.019353
Generation      1.558800
Legendary       0.186411
dtype: float64 std of the rest of the pokemon:  #             209.928799
Total         121.091682
HP             25.194299
Attack         3

Perform a hypothesis test comparing the mean of total points for water Pokemon to all non-water Pokemon. Assume the variances are equal. 

In [15]:
# H0: mu total stats for water pokemon = mu total stats for non-water pokemon
# H1: mu total stats for water pokemon != mu total stats for non-water pokemon

alpha = 0.05

water_pkmn = pokemon[pokemon['Type 1'] == 'Water']['Total']
non_water_pkmn = pokemon[pokemon['Type 1'] != 'Water']['Total']

stat, p_value = st.ttest_ind(water_pkmn, non_water_pkmn, equal_var=True)

print(stat)
print(p_value) 

-0.4418547448849676
0.6587140317488793


Write your conclusion below.

In [ ]:
# Our p_value is bigger than our alpha, so we can accept H0
# and start believing that the mu total stats for water pokemon
# are equal to the mu total stats for non-water pokemon.

# Challenge 2 - Matched Pairs Test

In this challenge we will compare dependent samples of data describing our Pokemon. Our goal is to see whether there is a significant difference between each Pokemon's defense and attack scores. Our hypothesis is that the defense and attack scores are equal. In the cell below, import the `ttest_rel` function from `scipy.stats` and compare the two columns to see if there is a statistically significant difference between them.

In [16]:
# H0: mu attack stats for each pokemon = mu defense stats for each pokemon
# H1: mu attack stats for each pokemon != mu defense stats for each pokemon

alpha = 0.05 

pokemon_atk = pokemon['Attack']
pokemon_def = pokemon['Defense']

stat, p_value = st.ttest_rel(pokemon_atk, pokemon_def) 

print(stat)
print(p_value) 

4.325566393330478
1.7140303479358558e-05


Describe the results of the test in the cell below.

In [ ]:
# Our p_value is smaller than our alpha, so we can reject H0
# and start believing that the mu attack stats for each pokemon
# are different to the mu defense stats for each pokemon.

We are also curious about whether therer is a significant difference between the mean of special defense and the mean of special attack. Perform the hypothesis test in the cell below. 

In [18]:
# H0: mu sp .attack stats for each pokemon = mu sp. defense stats for each pokemon
# H1: mu sp. attack stats for each pokemon != mu sp. defense stats for each pokemon

alpha = 0.05 

pokemon_spatk = pokemon['Sp. Atk']
pokemon_spdef = pokemon['Sp. Def']

stat, p_value = st.ttest_rel(pokemon_spatk, pokemon_spdef) 

print(stat)
print(p_value)


0.853986188453353
0.3933685997548122


Describe the results of the test in the cell below.

In [20]:
# p_value is bigger than alpha so we can accept h0 and
# start believing that mu sp atk and mu sp def are equal
# with a 95% confidence interval.

72.82
71.9025


As you may recall, a two sample matched pairs test can also be expressed as a one sample test of the difference between the two dependent columns.

Import the `ttest_1samp` function and perform a one sample t-test of the difference between defense and attack. Test the hypothesis that the difference between the means is zero. Confirm that the results of the test are the same.

In [22]:
# H0: mu attack stats for each pokemon -mu defense stats for each pokemon = 0
# H1: mu attack stats for each pokemon -mu defense stats for each pokemon != 0

alpha = 0.05 

sample = pokemon['Attack'] - pokemon['Defense']

stat, p_value = st.ttest_1samp(sample, 0) 

print(stat)
print(p_value) 

# The results are the same as the ones in the test performe before with
# Matched pairs set

4.325566393330478
1.7140303479358558e-05


# Bonus Challenge - The Chi-Square Test

The Chi-Square test is used to determine whether there is a statistically significant difference in frequencies. In other words, we are testing whether there is a relationship between categorical variables or rather when the variables are independent. This test is an alternative to Fisher's exact test and is used in scenarios where sample sizes are larger. However, with a large enough sample size, both tests produce similar results. Read more about the Chi Squared test [here](https://en.wikipedia.org/wiki/Chi-squared_test).

In the cell below, create a contingency table using `pd.crosstab` comparing whether a Pokemon is legenadary or not and whether the Type 1 of a Pokemon is water or not.

In [ ]:
# Your code here:



Perform a chi-squared test using the `chi2_contingency` function in `scipy.stats`. You can read the documentation of the function [here](https://docs.scipy.org/doc/scipy-0.15.1/reference/generated/scipy.stats.chi2_contingency.html).

In [ ]:
# Your code here:



Based on a 95% confidence, should we reject the null hypothesis?

In [ ]:
# Your answer here:

